In [2]:
import os
import requests
from ollama import chat
from ollama import ChatResponse
from firecrawl import FirecrawlApp
from pydantic import BaseModel
from typing import Optional
from dotenv import load_dotenv
from IPython.display import display, Markdown
from requests.exceptions import HTTPError
load_dotenv()

SERP_API_KEY = os.getenv("SERP_API_KEY")
FIRECRAWL_API_KEY = os.getenv("FIRECRAWL_API_KEY")
MODEL = os.getenv("MODEL")

In [3]:
def fetch_serp(q: str):
   url = 'https://serpapi.com/search'
   params = {
        'api_key': SERP_API_KEY,
        'engine': 'google',
        'q': q,
        'google_domain': 'google.com',
        'hl': 'en'
    }
   response = requests.get(url, params=params)
   if response.status_code == 200:
        json_response = response.json()
        organic = json_response['organic_results']
        formatted_results = []
        for result in organic:
            title = result['title']
            link = result['link']
            description = result['snippet']
            resource_object = {
                'title': title,
                'link': link,
                'description': description
            }
            formatted_results.append(resource_object)
        return formatted_results

In [4]:
app = FirecrawlApp(api_key=FIRECRAWL_API_KEY)
def retrieve_context(url:str):
    context = app.scrape_url(url, params={'formats':['markdown']})
    return context

In [5]:
class SearchQueries(BaseModel):
    queries: list[str] 

user_prompt = input("Topic of Research: ")
research_objective = input("Research objective: ")
research_width = input("Width of Research (1-5): ")

def generate_queries(prompt:str, objective:str, width:str):
    generated_queries: ChatResponse = chat(model=MODEL, messages=[
    {
        'role': 'user',
        'content': f"Given the user's prompt, generate {width} search queries (to be used as Google Search queries) that will get you started with performing deep research on the topic in hand. It is important that the queries you generate are unique and have ZERO overlap with each other. The number of search queries you are asked to generate defines the width of the user's research. Do not generate anything else apart from the {width} queries required. Here's the user's research title:{prompt}. And here's the research objective: {objective}. Your thought process while generating these queries should consider the main objective of the research, and you should figure out how to progressively.",
    },
    ],
    format=SearchQueries.model_json_schema())
    formatted_queries = SearchQueries.model_validate_json(generated_queries.message.content)
    return formatted_queries

In [6]:
search_results = []
current_queries = generate_queries(user_prompt, research_objective, research_width)
for q in current_queries.queries:
    results = fetch_serp(q)
    search_results.extend(results)
%store search_results
print(search_results)

Stored 'search_results' (list)
[{'title': 'Major Airlines of Australia/Oceania', 'link': 'https://www.nationsonline.org/oneworld/Airlines/airlines_oceania.htm', 'description': 'List of flag air carriers, domestic airlines, commercial airlines with passenger and cargo service, scheduled air carrier and low cost airlines.'}, {'title': 'List of Australian Airlines', 'link': 'https://www.thinkingaustralia.com/travel-to-australia/australian-airlines/', 'description': 'Australian Airlines · qantas · virgin · jetstar · tiger air · things you need to know · thinking australia? we can help! · We use cookies on Thinking Australia.'}, {'title': 'List of International & Domestic Airlines', 'link': 'https://au.wego.com/airlines', 'description': 'Popular Airlines in Australia ; Qantas \u200f(QF) ; Etihad Airways \u200f(EY) ; Jetstar Airways \u200f(JQ) ; Singapore Airlines \u200f(SQ) ; Regional Express Airlines \u200f(ZL).'}, {'title': 'Top Airlines and aviation companies in Australia', 'link': 'http

/Users/sanandh/Desktop/Projects/Deep-Research-Python/venv/lib/python3.12/site-packages/IPython/extensions/storemagic.py:229: UserWarning: This is now an optional IPython functionality, setting autorestore/search_results requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj


In [7]:
class ResultItem(BaseModel):
    title: str
    link: str

class RelevantQueries(BaseModel):
    r_queries: list[ResultItem]

references = []

def evaluate_relevance(prompt:str, objective:str, width:int, serp_results):
    relevant_queries: ChatResponse = chat(model=MODEL, messages=[
        {
            'role': 'user',
            'content': f"You are an expert data extractor. You are provided with a list of search results for a specific research query. This will be a list of objects containing the title, description and the link to an article/webpage. Use the title and description to verify if a particular search result is indeed relevant to the query. Return the titles AND links that are relevant to the research objective in the specified format. Here is the list of search results: {serp_results}. Here's the user's topic of research: {prompt}. The objective of the research is as follows: {objective}. You must return {width} results from the list that has been provided to you"
        }
    ],format=RelevantQueries.model_json_schema())
    formatted_relevant_queries = RelevantQueries.model_validate_json(relevant_queries.message.content)
    references.extend(formatted_relevant_queries.r_queries)
    return formatted_relevant_queries, references

In [8]:
research_context = []
evaluated_queries, _ = evaluate_relevance(user_prompt, research_objective, research_width, search_results)

In [9]:
evaluated_queries.r_queries

[ResultItem(title='Domestic airline competition in Australia | May 2024 report', link='https://www.accc.gov.au/system/files/domestic-airline-competition-in-australia-may-2024-report.pdf'),
 ResultItem(title='Australia Airlines Market Outlook to 2028 - Aviation', link='https://www.kenresearch.com/industry-reports/australia-airlines-market'),
 ResultItem(title="Qantas' brand health plummets; Virgin Australia sees gains ...", link='https://www.corporatetravelcommunity.com/qantas-brand-health-plummets-virgin-australia-sees-gains-in-consumer-sentiment/')]

In [10]:
for item in evaluated_queries.r_queries:
    print(f"Retrieving research context for the resource: {item.title}")
    try:
        c = retrieve_context(item.link)
        research_context.append(c.get("markdown", ""))
    except HTTPError as e:
        print(f"Failed to retrieve content from {item.title}: {str(e)}")
        references.remove(item)
        continue  # Skip to the next URL
    except Exception as e:
        print(f"Unexpected error while processing {item.title}: {str(e)}")
        continue  # Skip any other unexpected errors

%store research_context

Retrieving research context for the resource: Domestic airline competition in Australia | May 2024 report
Retrieving research context for the resource: Australia Airlines Market Outlook to 2028 - Aviation
Retrieving research context for the resource: Qantas' brand health plummets; Virgin Australia sees gains ...
Stored 'research_context' (list)


/Users/sanandh/Desktop/Projects/Deep-Research-Python/venv/lib/python3.12/site-packages/IPython/extensions/storemagic.py:229: UserWarning: This is now an optional IPython functionality, setting autorestore/research_context requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj


In [11]:
class ContextEval(BaseModel):
    further_research: bool
    probing_queries: Optional[SearchQueries]

def evaluate_context(prompt: str, objective: str, context: list[str], width: int) -> tuple[bool, Optional[list[str]]]:
    """Evaluates whether further research is needed based on the current context."""
    research_status: ChatResponse = chat(model=MODEL, messages=[
        {
            'role': 'user',
            'content': f"""
            You are an expert analyst. You are provided with information regarding the user's topic of research, which is: {prompt}.
            The objective of this research is: {objective}.
            The research context we have until now is here: {context}.
            Based on the given information, confirm if further research is required.
            However, if you think more information is required for a comprehensive report, generate {width} more probing questions
            that go deeper into the topic, but make sure the queries are specific and precise to the current context as they will be used
            to fetch Google search results, which in turn will be used for further research.
            """
        }
    ], format=ContextEval.model_json_schema())
    
    context_eval = ContextEval.model_validate_json(research_status.message.content)
    return context_eval.further_research, context_eval.probing_queries

In [12]:
def research_loop(prompt: str, objective: str, context: list[str], width: int) -> list[str]:
    """Manages the iterative research process to refine the research context."""
    max_iterations = 2  # Prevent infinite loops
    iteration_count = 0
    current_context = context.copy()
    
    while iteration_count < max_iterations:
        further_research, probing_queries = evaluate_context(prompt, objective, current_context, width)
        
        if not further_research:
            break
        
        iteration_count += 1
        new_search_results = []
        
        if probing_queries:  
            for q in probing_queries.queries:  
                new_search_results.extend(fetch_serp(q))
                
            current_evaluated_queries, refs = evaluate_relevance(prompt, objective, width, new_search_results)
            
            for _ in current_evaluated_queries.r_queries:
                print(f"Retrieving research context for the resource: {_.title}")
                try:
                    new_context = retrieve_context(_.link)
                    references.extend(_)
                    current_context.append(new_context.get("markdown", ""))
                except HTTPError as e:
                    print(f"Failed to retrieve content from {_.title}: {str(e)}")
                    references.remove(_)
                    print(f"Removed '{_}' from references")
                    continue
                except Exception as e:
                    print(f"Unexpected error while processing {_.title}: {str(e)}")
                    continue
    
    if iteration_count >= max_iterations:
        print("Max iterations reached. Proceeding with the available research context.")
    
    return current_context

In [13]:
def generate_report(prompt:str, objective:str, width: int, sources: list[str]) -> str:
    final_context = research_loop(prompt, objective, research_context, width)
    final_report: ChatResponse = chat(model=MODEL, messages=[
        {
            'role': 'user',
            'content':f"""You are an expert analyst. Your job is to use the provided research prompt, objective and context to generate a clear and comprehensive report.
            Research Title: {prompt}
            Research Objective: {objective}
            Research Context: {final_context}
            With this information, generate a detailed report as instructed. Be sure to include all sources, persons, objects etc in the report. You MUST include a references section and appropriately add citations, the references are here: {sources}. The report must be in markdown format. You are allowed to take liberties with section titles and in-report formatting. Ensure that the report itself adheres to the user's requirements and does not deviate away from the research's goals. You MUST remain aligned to the research title and objective, even if the context provided contains information that may cover aspects other than the ones mentioned by the user. Use the research title provided by the user for the report's title and follow the instructions as mentioned above."""
        }
    ])
    return final_report.message.content

In [14]:
report = generate_report(user_prompt,research_objective, research_width, references)

In [15]:
display(Markdown(report))

# Impact of Recent Events on Consumer Sentiment Towards Qantas and Virgin Australia

## Introduction

The airline industry in Australia has seen significant shifts in consumer sentiment, particularly between two major players: Qantas and Virgin Australia. This report examines how recent events have impacted the brand health of these airlines, drawing from various sources to provide a comprehensive analysis.

## Analysis of Recent Events

### Decline in Qantas' Brand Health

Qantas has experienced a notable decline in brand health due to several key events:

1. **Legal Issues**: The airline faced legal challenges, including being fined for misleading claims about its carbon emissions. This incident raised concerns about transparency and corporate responsibility.

2. **Operational Problems**: Frequent delays and cancellations have plagued Qantas flights, leading to customer dissatisfaction.

3. **Labor Disputes**: Ongoing disputes with pilots over pay and conditions have resulted in industrial action, further tarnishing the airline's image.

4. **Financial Performance**: A reported $1 billion loss exacerbated the negative perception of Qantas among consumers.

### Gains for Virgin Australia

In contrast, Virgin Australia has seen an improvement in consumer sentiment:

1. **Operational Reliability**: The airline has maintained a reputation for operational reliability and punctuality, enhancing customer satisfaction.

2. **Customer Service**: Positive feedback regarding customer service and transparency has bolstered Virgin's brand image.

3. **Competitive Pricing**: Offering competitive pricing strategies has attracted cost-conscious travelers, contributing to increased market share.

## Comparative Analysis

### Brand Health Metrics

According to a report by the Corporate Travel Community, Qantas' brand health has significantly declined, while Virgin Australia has experienced gains in consumer sentiment ([Qantas' brand health plummets; Virgin Australia sees gains...](https://www.corporatetravelcommunity.com/qantas-brand-health-plummets-virgin-australia-sees-gains-in-consumer-sentiment/)).

### Market Competition

The Australian Competition and Consumer Commission (ACCC) highlights the competitive dynamics between Qantas and Virgin Australia, noting that Virgin's strategic improvements have positioned it as a formidable competitor ([Domestic airline competition in Australia | May 2024 report](https://www.accc.gov.au/system/files/domestic-airline-competition-in-australia-may-2024-report.pdf)).

## Conclusion

The recent events surrounding Qantas and Virgin Australia illustrate the impact of operational, financial, and reputational factors on consumer sentiment. While Qantas faces challenges in restoring its brand health, Virgin Australia's strategic initiatives have positively influenced its market position. Continued monitoring of these developments will be essential for understanding future trends in the Australian airline industry.

## References

- [Domestic airline competition in Australia | May 2024 report](https://www.accc.gov.au/system/files/domestic-airline-competition-in-australia-may-2024-report.pdf)
- [Australia Airlines Market Outlook to 2028 - Aviation](https://www.kenresearch.com/industry-reports/australia-airlines-market)
- [Qantas' brand health plummets; Virgin Australia sees gains ...](https://www.corporatetravelcommunity.com/qantas-brand-health-plummets-virgin-australia-sees-gains-in-consumer-sentiment/)

In [17]:
with open('research_report.md', 'w', encoding='utf-8') as f:
    f.write(report)